In [7]:
import pymongo
import json
import os
import tqdm
import re
import pickle
import openai
import tiktoken
from langchain.docstore.document import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
from deep_translator import GoogleTranslator # До 5000 символов за раз

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
transtator = GoogleTranslator(source='ru', target='en')

In [2]:
#@title Использую БД MongoDB (локально)
# Полезные короткие инструкции для работы с MongoDB:
# Создание локальной базы MongoDB: https://www.youtube.com/watch?v=pmjHPOPwX2A
# Pymongo: https://www.youtube.com/playlist?list=PLEYdORdflM3ltl8E2D1XLT3Fe2_e9eEdd

client_mongo = pymongo.MongoClient('mongodb://localhost:27017/') # Соединение с базой
db_hr = client_mongo.hr # подключаюсь к БД "hr", если такой БД нет, будет создана
vacancies_collection = db_hr.vacancies # подключаю Коллекцию "vacancies" в базе "hr", если коллекции нет, будет создана
resumes_collection = db_hr.resumes # подключаю Коллекцию "resumes" в базе "hr", если коллекции нет, будет создана

In [2]:
# Записываю в коллекцию vacancies базы hr все вакансии из файлов json
for filename in os.listdir('../Vacancies'): # Локальная папка с вакансиями
    if filename.endswith('.json'):
        with open(os.path.join('../Vacancies', filename), 'r', encoding='utf-8') as v:
            vacancy_dict = json.load(v)
            vacancy_dict['_id'] = filename # Добавляю поле "_id" с именем файла для каждой вакансии
            try:
                vacancies_collection.insert_one(vacancy_dict)
            except Exception as ex:
                pass

# Записываю в коллекцию resumes базы hr все резюме из файлов json
for filename in os.listdir('../Resumes'): # Локальная папка с резюме
    if filename.endswith('.json'):
        with open(os.path.join('../Resumes', filename), 'r', encoding='utf-8') as r:
            resume_dict = json.load(r)
            resume_dict['_id'] = filename # Добавляю поле "_id" с именем файла для каждого резюме
            try:
                resumes_collection.insert_one(resume_dict)
            except Exception as ex:
                pass

print(f'К-во вакансий в базе данных: {vacancies_collection.count_documents({})}. '
      f'\nК-во резюме в базе данных: {resumes_collection.count_documents({})}.')

К-во вакансий в базе данных: 100. 
К-во резюме в базе данных: 100.


# Вакансии
### Ключевые поля:
- Позиция "position"
- Уровень "experienceLevels"
- Формат работы "workFormat"
- Локация "requiredLocation"
- Зарплата "partnerRates" в час (168 часов в месяце)
#### Три поля по скилам:
- Скилы "skills"
- Требования "mandatoryRequirements"
- Доп. требования "additionalRequirements"

In [4]:
# Формирую строки из ключевых полей
for value in tqdm.tqdm(vacancies_collection.find()): # по всем вакансиям из базы, коллекции vacancies
    try: # Если указана зарплата (за час)
        salary = int(re.findall(r'\d+', f"{value['data']['partnerRates']}")[0]) * 168 # Зарплата за 168 часов в месяц
        salary = f"Salary: {salary} рублей в месяц."
    except:
        salary = ''

    try: # Если указаны Add.Requirements
        add_requirements = f"Add.Requirements: {(' ').join(value['data']['additionalRequirements'])}"
    except:
        add_requirements = ''

    vacancy_line = re.sub(r'\s+', ' ', f"""
                   Position: {value['data']['position']}.
                   Experience Levels: {(', ').join(value['data']['experienceLevels'])}.
                   Work Format: {value['data']['workFormat']}.
                   Required Location: {value['data']['requiredLocation']}.
                   {salary}
                   Skills: {(', ').join(value['skills'])}.
                   Requirements: {(' ').join(value['data']['mandatoryRequirements'])}
                   {add_requirements}""")

    vacancy_line_en = transtator.translate(vacancy_line) # Английский
    try: # сохраняю сформированные строки из полей вакансий в коллекцию vacancies
        vacancies_collection.find_one_and_update({'_id': value['_id']}, # нахожу вакансию по _id
                                                 {'$set': {'line_first': vacancy_line,
                                                           'line_first_en': vacancy_line_en}})
    except Exception as ex:
        print(ex)

# Пример строки с выбранными ключевыми полями
ex_vacancy = vacancies_collection.find_one({'_id': '1С_разработчик-00e46f89-176b-4447-beaf-80a7a63fb072.json'})
ex_vacancy['line_first']

100it [01:31,  1.09it/s]


' Position: 1С разработчик. Experience Levels: Middle, Middle+, Senior. Work Format: Удаленный. Required Location: Любая. Salary: 403200 рублей в месяц. Skills: 1C: Production, 1C: Salary and staff, 1C: Management of the holding, 1C, 1C: ERP, Remote work, 1C ERP, 1C: Accounting and warehouse, C., MS Excel, Excel, The column, IT Recruitment, Full Stack, Management of requirements, Test case, The Auto Grid, Analysis of the requirements, The test, Creating the 1C configuration, 1C programming, 1C: Salaries and personnel, Leadership of the team of developers, Testing, The energy stack. Requirements: Требуется 1С разработчики, консультанты и функциональные архитекторы; Вся информация по стекам, грейдам и дополнительным требованиям указана в Excel файле, в разделе "Тестовое задание"; При отклике на вакансию нужно указывать ID должности, в файле данный столбец выделен желтым. '

In [6]:
#@title Формирую чанки (Langchain Document)
vacancies_fields_first_chunks_en = [] # Список для чанков
for value in vacancies_collection.find(): # по всем вакансиям из базы, коллекции vacancies
    vacancies_fields_first_chunks_en.append(Document(page_content=value['line_first_en'],
                                                     metadata={'file': value['_id']}))
# Пример чанка
vacancies_fields_first_chunks_en[0]

Document(page_content='Position: 1C developer. Experience Levels: Middle, Middle+, Senior. Work Format: Remote. Required Location: Any. Salary: 403,200 rubles per month. Skills: 1C: Production, 1C: Salary and staff, 1C: Management of the holding, 1C, 1C: ERP, Remote work, 1C ERP, 1C: Accounting and warehouse, C., MS Excel, Excel, The column, IT Recruitment , Full Stack, Management of requirements, Test case, The Auto Grid, Analysis of the requirements, The test, Creating the 1C configuration, 1C programming, 1C: Salaries and personnel, Leadership of the team of developers, Testing, The energy stack. Requirements: 1C developers, consultants and functional architects required; All information on stacks, grades and additional requirements is indicated in the Excel file, in the “Test task” section; When responding to a vacancy, you must indicate the position ID; in the file, this column is highlighted in yellow.', metadata={'file': '1С_разработчик-00e46f89-176b-4447-beaf-80a7a63fb072.json'

In [14]:
#@title Индексная (векторная) база по выбранным полям Вкансий
TOTAL_AMOUNT = 0
# Подсчет токенов
def num_tokens_from_string(string, encoding_name):
    encoding = tiktoken.get_encoding(encoding_name)
    return len(encoding.encode(string))

# количество токенов индексной базы и стоимость. model Ada v2
def tokens_count_db_index(source_chunks):
    global TOTAL_AMOUNT
    count_tokens = 0
    count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
    count_tokens += count_token
    # Embedding model Ada v2 - $0.10 / 1M tokens - 13/03/2024 - https://openai.com/pricing
    price = 0.1 * count_tokens / 1e6
    TOTAL_AMOUNT += price
    print(f'К-во токенов в документе: {count_token}. '
          f'Цена создания индексной базы: $ {price}')


# Создаю индексную (Векторную) базу и сохраняю
# Если не в колабе, ВКЛЮЧИТЬ VPN
db_first_vacancies = FAISS.from_documents(vacancies_fields_first_chunks_en, OpenAIEmbeddings())
db_first_vacancies.save_local(folder_path='../data', index_name='vacancies_fields_first_db_index')

tokens_count_db_index(vacancies_fields_first_chunks_en)
print(f'К-во записей в индексной базе: {len(db_first_vacancies.docstore._dict)}')

К-во токенов в документе: 45494. Цена создания индексной базы: $ 0.0045494
К-во записей в индексной базе: 100


In [18]:
# Пример чанка из индексной базы
db_first_vacancies.docstore._dict['afbbe803-6b33-43af-bce8-71d6df6db7dc']

Document(page_content='Position: 1C developer. Experience Levels: Middle, Middle+, Senior. Work Format: Remote. Required Location: Any. Salary: 403,200 rubles per month. Skills: 1C: Production, 1C: Salary and staff, 1C: Management of the holding, 1C, 1C: ERP, Remote work, 1C ERP, 1C: Accounting and warehouse, C., MS Excel, Excel, The column, IT Recruitment , Full Stack, Management of requirements, Test case, The Auto Grid, Analysis of the requirements, The test, Creating the 1C configuration, 1C programming, 1C: Salaries and personnel, Leadership of the team of developers, Testing, The energy stack. Requirements: 1C developers, consultants and functional architects required; All information on stacks, grades and additional requirements is indicated in the Excel file, in the “Test task” section; When responding to a vacancy, you must indicate the position ID; in the file, this column is highlighted in yellow.', metadata={'file': '1С_разработчик-00e46f89-176b-4447-beaf-80a7a63fb072.json'

# Резюме
### Ключевые поля резюме для поиска вакансий
- Позиция "title" + "professional_roles.name"
- Скилы "skill_set"
- Доп. информация от кандидата "skills"
- Зарплата "salary"
- График работы "schedule"
- Локация "area.name"
- Отношение к релокации "relocation"
- Опыт работы "experience.position" и "experience.description"
- Трудовой стаж (месяцев) "total_experience"
- Языки "language"

In [3]:
# Формирую строки из ключевых полей резюме
for value in resumes_collection.find(): # по все резюме из коллекции
    schedules = '' # График работы
    for schedule in value["schedules"]:
        schedules += f"{schedule['name']}, "

    experience = '' # Опыт
    for n, place in enumerate(value['experience']):
        experience += f"Job experience {n+1}: Position - {place['position']}. \
                             Job description {n+1}: {place['description']} "

    languages = '' # Языки
    for lang in value['language']:
        languages += f"{lang['name']} - level: {lang['level']['name']}. "

    try: # Если указана зарплата
        currency = value['salary']['currency']
        salary = f"Desired salary: {value['salary']['amount']} {currency if currency!='RUR' else 'рублей в месяц'}. "
    except Exception as ex:
        salary = ''

    if value['skills'] != None: # Add.skills
        add_skills = f"Add.skills: {value['skills']}."
    else:
        add_skills = ''


    # 1-й блок резюме
    resume_line_1 = re.sub(r'\s+', ' ', f"""
                         Position: {value['title']}. {value['professional_roles'][0]['name']}.
                         Skills: {(', ').join(value['skill_set'])}.
                         {add_skills}""")
    # 2- блок резюме
    resume_line_2 = re.sub(r'\s+', ' ', f"""
                         {salary}
                         Languages: {languages}.
                         Job schedule: {schedules}.
                         Location: {value['area']['name']}.
                         Attitude to relocation: {value['relocation']['type']['name']}.
                         Total experience: {value['total_experience']['months']} months. """)
    # 3-й блок резюме
    resume_line_3 = re.sub(r'\s+', ' ', f'{experience}')
    try: # сохраняю сформированные строки из полей резюме в коллекцию resumes
        resumes_collection.find_one_and_update({'_id': value['_id']}, # нахожу резюме по _id
                                               {'$set': {'line_first_1': resume_line_1,    # Position, Skills, Add.skills
                                                         'line_first_2': resume_line_2,    # Salary, Lang, Schedule, Location...
                                                         'line_first_3': resume_line_3}})  # Experience (position, description)
    except Exception as ex:
        print(ex)

In [4]:
# Пример строки с выбранными ключевыми полями
ex_resume = resumes_collection.find_one({'_id': 'b1dcc297-a293-405b-8e46-f47328bea37a.json'})
print(ex_resume['line_first_1'], '\n')
print(ex_resume['line_first_2'], '\n')
print(ex_resume['line_first_3'])

 Position: Java программист. Программист, разработчик. Skills: ORACLE, Docker, База данных: Oracle, MySQL, REST, XML, СУБД, Atlassian Jira, Gradle, JDBC, Spring, MongoDB, Unix, GitHub, Hibernate, JSON, REST API, HTTP, Unit Testing, Backend, Maven, JPA, JMS, SOAP, Microservices, Java Script, Java EE, Java SE, MS SQL, Linux.  

 Desired salary: 200000 рублей в месяц. Languages: Русский - level: Родной. Английский - level: B1 — Средний. . Job schedule: Полный день, Сменный график, Гибкий график, Удаленная работа, . Location: Оренбург. Attitude to relocation: не готов к переезду. Total experience: 364 months.  

Job experience 1: Position - Java developer (удаленно). Job description 1: Разработка приложений с применением Java 8/11/17, Spring (Data, Boot, Security, Web, Cloud Config, AOP), Git / Gitflow / Bitbucket, Jenkins, микросервисы, docker / docker-compose, Maven / Gradle, Oracle / MySQL / PostgreSQL / MS SQL, Hibernate, JBoss / Glassfish / WildFly / Tomcat / Azure, Unix (FreeBSD, Red

# Подбор вакансий (similarity_search_with_score) под резюме

In [3]:
#@title similarity_search_with_score
def ids_and_scores(query, db):
    docs_and_scores = db.similarity_search_with_score(query, k=5)
    print('\nScores:', [doc[1] for doc in docs_and_scores])
    print(', '.join([f"\n_id {i + 1}: {doc[0].metadata['file']}"
                     for i, doc in enumerate(docs_and_scores)]))
    print('-'*80)
    return [f"{doc[0].metadata['file']}" for doc in docs_and_scores]

# Загружаю ранее сохраненную индексную базу с вакансиями
db_first_vacancies = FAISS.load_local(folder_path='../data',
                           embeddings=OpenAIEmbeddings(),
                           index_name='vacancies_fields_first_db_index')

In [7]:
# Любое резюме
ex_resume = resumes_collection.find_one({'_id': 'b1dcc297-a293-405b-8e46-f47328bea37a.json'})
query_1 = transtator.translate(ex_resume['line_first_1'])
query_2 = transtator.translate(ex_resume['line_first_2'])
query_3 = transtator.translate(ex_resume['line_first_3'][:4999]) # переводчик до 5000 символов

ids_1 = ids_and_scores(query_1, db_first_vacancies)
ids_2 = ids_and_scores(query_1 + query_2, db_first_vacancies)
ids_3 = ids_and_scores(query_1 + query_2 + query_3, db_first_vacancies)


Scores: [0.23593657, 0.25759953, 0.2642653, 0.26903987, 0.28311896]

_id 1: Java_разработчик-2ce1bca2-5a23-4f68-8566-8ff0f7f87e3c.json, 
_id 2: Java_разработчик-6e8ea5e3-47fb-4a15-84fa-6ab618dcbbe0.json, 
_id 3: Java_разработчик-f6b48d5d-882e-4afd-8826-3d46b78342c6.json, 
_id 4: Java_разработчик-654b79d7-6b07-4705-969d-41a9eef10503.json, 
_id 5: Backend_разработчик-bb3c56dc-e54f-4dba-bfa1-6f45e42ff1ab.json
--------------------------------------------------------------------------------

Scores: [0.13272537, 0.14302576, 0.1458421, 0.15256974, 0.18882027]

_id 1: Java_разработчик-2ce1bca2-5a23-4f68-8566-8ff0f7f87e3c.json, 
_id 2: Java_разработчик-f6b48d5d-882e-4afd-8826-3d46b78342c6.json, 
_id 3: Java_разработчик-654b79d7-6b07-4705-969d-41a9eef10503.json, 
_id 4: Java_разработчик-6e8ea5e3-47fb-4a15-84fa-6ab618dcbbe0.json, 
_id 5: Backend_разработчик-8a68226f-ea25-4b81-8eee-a0e8ea99fb2d.json
--------------------------------------------------------------------------------

Scores: [0.1206

In [4]:
# similarity_search_by_vector
ex_resume = resumes_collection.find_one({'_id': 'b1dcc297-a293-405b-8e46-f47328bea37a.json'})
query_1 = transtator.translate(ex_resume['line_first_1'])
query_2 = transtator.translate(ex_resume['line_first_2'])
query_3 = transtator.translate(ex_resume['line_first_3'][:4999]) # переводчик до 5000 символов

embedding_vector = OpenAIEmbeddings().embed_query(query_1 + query_2 + query_3)
docs = db_first_vacancies.similarity_search_by_vector(embedding_vector, k=5)
[doc.metadata for doc in docs]

[{'file': 'Java_разработчик-2ce1bca2-5a23-4f68-8566-8ff0f7f87e3c.json'},
 {'file': 'Java_разработчик-6e8ea5e3-47fb-4a15-84fa-6ab618dcbbe0.json'},
 {'file': 'Java_разработчик-f6b48d5d-882e-4afd-8826-3d46b78342c6.json'},
 {'file': 'Java_разработчик-654b79d7-6b07-4705-969d-41a9eef10503.json'},
 {'file': 'Backend_разработчик-8a68226f-ea25-4b81-8eee-a0e8ea99fb2d.json'}]

# OpenAI

In [8]:
# функция добавления переходов на новую строку для удобства чтения
def format_newlines(text, max_len=100):
    lines = text.splitlines()
    new_lines = []
    for line in lines:
        words = line.split()
        current_line = ""
        for word in words:
            if len(current_line + " " + word) > max_len:
                new_lines.append(current_line)
                current_line = ""
            current_line += f' {word}'
        new_lines.append(current_line)
    return "\n".join(new_lines)

# Стоимость запроса + ответ для "gpt-3.5-turbo-0125"
def print_tokens_count(completion):
    # "gpt-3.5-turbo-0125" - Input: $0.50 / 1M tokens - Output: $1.50 / 1M tokens - 07/03/2024 - https://openai.com/pricing
    price = 0.5 * completion.usage.prompt_tokens / 1e6 + 1.5 * completion.usage.completion_tokens / 1e6
    print(f'Использовано токенов: {completion.usage.prompt_tokens} + '
                                f'{completion.usage.completion_tokens} = '
                                f'{completion.usage.total_tokens}. '
                                f'Цена запроса + ответ: $ {price}\n')
    # global TOTAL_AMOUNT
    # TOTAL_AMOUNT += price

def answer_gpt(messages, temp=0.5):
    completion = openai.chat.completions.create(model="gpt-3.5-turbo-0125",
                                                messages=messages,
                                                temperature=temp)
    print_tokens_count(completion)
    return completion.choices[0].message.content

In [9]:
# Резюме: 'b1dcc297-a293-405b-8e46-f47328bea37a.json'
test_resume = query_1 + query_2 + query_3
# Выбранная вакасия: 'Java_разработчик-2ce1bca2-5a23-4f68-8566-8ff0f7f87e3c.json'
test_vacancy = vacancies_collection.find_one(
               {'_id': 'Java_разработчик-2ce1bca2-5a23-4f68-8566-8ff0f7f87e3c.json'})['line_first_en']

system = re.sub(r'\s+', ' ', """You are a professional HR.
    You help candidates (applicants) to get a job for the chosen vacancy""")

message_content = re.sub(r'\s+', ' ', f"""Based on the candidate's resume, you have found a suitable vacancy.
    Candidate's desired position, candidate's skills, additional information about the candidate
    from the resume: {test_resume}. Requirements for the candidate, according
    to the vacancy you have chosen: {test_vacancy}. Analyze the information from the candidate's
    resume and the requirements from the vacancy.""")

question = re.sub(r'\s+', ' ', """Answer the questions:
    1. What is the percentage from 0 to 100 of matching skills in the candidate's resume compared
       to the necessary skills in the vacancy?
    2. What skills and competencies does the candidate lack in the resume compared to the necessary
       skills and competencies in the vacancy?
    3. What can be said about other requirements from the vacancy (except skills) to the candidate
       compared with the candidate's competencies (except skills)?

    Answer all questions in Russian, with the exception of the words denoting the candidate's skills,
    if you write about them. So, answer in Russian, but if you use words denoting the candidate's skills,
    write skills in English.""")

message = [{"role": "system", "content": system},
           {"role": "user", "content": f"{message_content}\n{question}"}]

response = answer_gpt(message, temp=0.3)
print(format_newlines(response))

Использовано токенов: 1217 + 356 = 1573. Цена запроса + ответ: $ 0.0011424999999999999

 1. Процент совпадения навыков кандидата из резюме с необходимыми навыками в вакансии составляет
 около 70%.

 2. Кандидату из резюме не хватает следующих навыков и компетенций по сравнению с необходимыми
 навыками и компетенциями в вакансии: Kotlin, Spring Cloud, *nix command line, log aggregation
 systems (ELK), Kubernetes, ability to analyze performance problems, experience in designing and
 implementing CI/CD processes on Gitlab/analogues, experience working in a team using agile
 development methods (Scrum, Kanban) и опыт внедрения проектов в области FinTech.

 3. По другим требованиям из вакансии (кроме навыков) кандидат соответствует следующим компетенциям:
 гибкое мышление, работа в команде, гибкость, гибкость мышления, гибкость методологии, работа в
 проекте, работа в команде, командное управление, формирование команды, управление командой, работа
 в команде, гибкость, работа в команде, про